In [1]:
import pandas as pd
import numpy as np

data = "/mnt/d/Data/mangaki-data-challenge/"

record = pd.read_csv(data+'watched.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': 'category'
})

train_full = pd.read_csv(data+'train_withcv.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16,
    'rating': np.int8,
    'cv': np.int8
})

test = pd.read_csv(data+'test.csv', dtype={
    'user_id': np.int16,
    'work_id': np.int16
})

In [2]:
u1 = pd.crosstab(record.user_id, record.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)

In [3]:
ratelut = {
    'dislike':1.0,
    'like':3.0,
    'love':4.0,
    'neutral':2.0
}
record['score']=record.rating.map(lambda x: ratelut[x]).astype(float)
u2 = record[['user_id', 'score']].groupby(by='user_id')['score'].agg(['mean', 'std']).rename(columns={'mean':'user_mean', 'std':'user_std'})

In [4]:
i1 = pd.crosstab(record.work_id, record.rating).add_prefix('work_').apply(lambda r: r/r.sum(), axis=1)
i2 = record[['work_id', 'score']].groupby(by='work_id')['score'].agg(['mean', 'std']).add_prefix('item_')

In [7]:
# deprecated
valid = train_full[train_full.cv==3]
train = train_full[~train_full.cv.isin([3])]
u3 = pd.crosstab(train.user_id, train.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)
i3 = pd.crosstab(train.work_id, train.rating).add_prefix('item_').apply(lambda r: r/r.sum(), axis=1)

In [9]:
valid.shape

(3754, 4)

In [10]:
for cv in [1,2,3]:
    valid = train_full[train_full.cv==cv]
    train = train_full[~train_full.cv.isin([cv])]
    train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
    valid = valid.merge(u1, left_on='user_id', right_index=True, how='left').\
                  merge(u2, left_on='user_id', right_index=True, how='left').\
                  merge(i1, left_on='work_id', right_index=True, how='left').\
                  merge(i2, left_on='work_id', right_index=True, how='left')
    train = train.drop(['cv'], axis=1)
    train.to_csv(data+"train_{0}.csv".format(cv), index=False)
    valid = valid.drop(['cv'], axis=1)
    valid.to_csv(data+"valid_{0}.csv".format(cv), index=False)

In [27]:
train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')

In [28]:
train = train.drop(['cv'], axis=1)
train.to_csv(data+"train_1.csv", index=False)

In [20]:
valid = valid.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')
valid = valid.drop(['cv'], axis=1)

In [21]:
valid.to_csv(data+"valid_3.csv", index=False)

In [11]:
train = train_full
#u3 = pd.crosstab(train.user_id, train.rating).add_prefix('user_').apply(lambda r: r/r.sum(), axis=1)
#i3 = pd.crosstab(train.work_id, train.rating).add_prefix('item_').apply(lambda r: r/r.sum(), axis=1)
train = train.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')
train = train.drop(['cv'], axis=1)
train.to_csv(data+"baseline/train_0.csv", index=False)

In [12]:
test = test.merge(u1, left_on='user_id', right_index=True, how='left').\
merge(u2, left_on='user_id', right_index=True, how='left').\
merge(i1, left_on='work_id', right_index=True, how='left').\
merge(i2, left_on='work_id', right_index=True, how='left')
test.to_csv(data+"baseline/test_0.csv", index=False)

W2V feature

In [22]:
w2vdim=10

In [23]:
from gensim.models.word2vec import Word2Vec
from random import sample

record_positive = record[record.rating.isin(['like', 'love'])]
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_positive.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

item_w2v_positive = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_positive:
        item_index.append(item)
        item_w2v_positive.append(w2v_positive['i{0}'.format(item)])
item_w2v_positive = np.vstack(item_w2v_positive)
i4 = pd.DataFrame(data = item_w2v_positive, index=item_index, columns=['itemw2vpos_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/item_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [24]:
# for user
record_positive = record[record.rating.isin(['like', 'love'])]
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_positive.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_positive = Word2Vec(sentences, size=w2vdim)

user_w2v_positive = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_positive:
        user_index.append(user)
        user_w2v_positive.append(w2v_positive['u{0}'.format(user)])
user_w2v_positive = np.vstack(user_w2v_positive)
u4 = pd.DataFrame(data = user_w2v_positive, index=user_index, columns=['userw2vpos_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/user_w2v_shuffled_{0}d.csv".format(w2vdim), index=False)

In [25]:
from gensim.models.word2vec import Word2Vec

record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveuser = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_negative_effectiveuser = record_negative_effectiveuser[record_negative_effectiveuser.work_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    t = ['i{0}'.format(str(i)) for i in itm[1].work_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_negative = Word2Vec(sentences, size=w2vdim)

item_w2v_negative = []
item_index = []
for item in range(9897):
    if 'i{0}'.format(item) in w2v_negative:
        item_index.append(item)
        item_w2v_negative.append(w2v_negative['i{0}'.format(item)])
item_w2v_negative = np.vstack(item_w2v_negative)
i4 = pd.DataFrame(data = item_w2v_negative, index=item_index, columns=['itemw2vpos_{0}'.format(i) for i in range(w2vdim)])
i4.index.name='work_id'

i4.reset_index().to_csv(data+"features/item_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

In [26]:
record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveitem = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_negative_effectiveitem = record_negative_effectiveitem[record_negative_effectiveitem.user_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    t = ['u{0}'.format(str(i)) for i in itm[1].user_list]
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    sentences.append(sample(t, len(t)))
    
w2v_negative = Word2Vec(sentences, size=w2vdim)

user_w2v_negative = []
user_index = []
for user in range(1983):
    if 'u{0}'.format(user) in w2v_negative:
        user_index.append(user)
        user_w2v_negative.append(w2v_negative['u{0}'.format(user)])
user_w2v_negative = np.vstack(user_w2v_negative)
u4 = pd.DataFrame(data = user_w2v_negative, index=user_index, columns=['userw2vpos_{0}'.format(i) for i in range(w2vdim)])
u4.index.name='user_id'

u4.reset_index().to_csv(data+"features/user_w2vneg_shuffled_{0}d.csv".format(w2vdim), index=False)

LDA feature

In [25]:
num_topics = 10

In [26]:
from gensim.models.ldamodel import LdaModel

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5}
for itm in record_positive.iterrows():
    for work in itm[1].work_id:
        strwork = 'i{0}'.format(work)
        if strwork not in dictionary:
            dictionary[strwork]=cnt
            cnt+=1
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lda_user = LdaModel(corpus=corpus, num_topics=num_topics, id2word=invdict)

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating) if 'i{0}'.format(i) in dictionary])

user_lda_positive = []
for i, topic in enumerate(lda_user[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lda_positive.append(temp)
user_lda_positive = np.vstack(user_lda_positive)
u5 = pd.DataFrame(data = user_lda_positive, index=record_positive.index, columns=['user_ldapos_{0}'.format(i) for i in range(num_topics)])

u5.reset_index().to_csv(data+"features/user_lda_{0}d.csv".format(num_topics), index=False)

In [27]:
record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5}
for itm in record_positive.iterrows():
    for user in itm[1].user_id:
        struser = 'u{0}'.format(user)
        if struser not in dictionary:
            dictionary[struser]=cnt
            cnt+=1
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lda_item = LdaModel(corpus=corpus, num_topics=num_topics, id2word=invdict)

record_positive = record[record.rating.isin(['like', 'love', 'neutral'])]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating) if 'u{0}'.format(i) in dictionary])

item_lda_positive = []
for i, topic in enumerate(lda_item[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lda_positive.append(temp)
item_lda_positive = np.vstack(item_lda_positive)
i5 = pd.DataFrame(data = item_lda_positive, index=record_positive.index, columns=['item_ldapos_{0}'.format(i) for i in range(num_topics)])

i5.reset_index().to_csv(data+"features/item_lda_{0}d.csv".format(num_topics), index=False)

In [31]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveuser = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_negative_effectiveuser = record_negative_effectiveuser[record_negative_effectiveuser.work_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveuser), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['i{0}'.format(str(i)) for i in itm[1].work_list])
    
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(rc) for rc in sentences]

lda_user = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

record_negative = record[record.rating.isin(['dislike'])]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_list': lambda x: list(x)})

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['i{0}'.format(str(i)) for i in itm[1].work_list])
corpus = [dictionary.doc2bow(rc) for rc in sentences]

user_lda_negative = []
for i, topic in enumerate(lda_user[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lda_negative.append(temp)
user_lda_negative = np.vstack(user_lda_negative)
u5 = pd.DataFrame(data = user_lda_negative, index=record_negative.index, columns=['user_ldapos_{0}'.format(i) for i in range(num_topics)])

u5.reset_index().to_csv(data+"features/user_ldaneg_{0}d.csv".format(num_topics), index=False)

In [32]:
record_negative = record[record.rating.isin(['dislike'])]
record_negative_effectiveitem = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_negative_effectiveitem = record_negative_effectiveitem[record_negative_effectiveitem.user_count>5]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})
record_negative = pd.concat((record_negative, record_negative_effectiveitem), axis=1, join='inner')

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['u{0}'.format(str(i)) for i in itm[1].user_list])
    
dictionary = corpora.Dictionary(sentences)
corpus = [dictionary.doc2bow(rc) for rc in sentences]

lda_item = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)

record_negative = record[record.rating.isin(['like', 'love'])]
record_negative = record_negative[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_list': lambda x: list(x)})

sentences = []
for itm in record_negative.iterrows():
    sentences.append(['u{0}'.format(str(i)) for i in itm[1].user_list])
corpus = [dictionary.doc2bow(rc) for rc in sentences]

item_lda_negative = []
for i, topic in enumerate(lda_item[corpus]):
    temp = np.zeros(num_topics)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lda_negative.append(temp)
item_lda_negative = np.vstack(item_lda_negative)
i5 = pd.DataFrame(data = item_lda_negative, index=record_negative.index, columns=['item_ldapos_{0}'.format(i) for i in range(num_topics)])

i5.reset_index().to_csv(data+"features/item_ldaneg_{0}d.csv".format(num_topics), index=False)

convert to libfm format

In [55]:
from scipy.sparse import coo_matrix, hstack
from sklearn.datasets import dump_svmlight_file


path = "/mnt/d/Data/mangaki-data-challenge/"
for cv in [1,2,3]:
    train = pd.read_csv(path+"baseline/train_{0}.csv".format(cv))
    valid = pd.read_csv(path+"baseline/valid_{0}.csv".format(cv))
    
    data = []
    row = []
    col = []
    for i, itm in enumerate(train.iterrows()):
        row.append(i)
        col.append(itm[1].user_id)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].work_id+1983)
        data.append(1.0)

    basic = coo_matrix((data, (row, col)), shape=(train.shape[0], 1983+9897))
    ext = train.ix[:, 3:].fillna(0).values
    ext[:, 4] = ext[:, 4]/4
    ext[:, 10] = ext[:, 10]/4
    value = hstack([basic, ext])
    dump_svmlight_file(value, train.rating.values, path+"libfm/train_{0}.csv".format(cv))
    
    data = []
    row = []
    col = []
    for i, itm in enumerate(valid.iterrows()):
        row.append(i)
        col.append(itm[1].user_id)
        data.append(1.0)
        row.append(i)
        col.append(itm[1].work_id+1983)
        data.append(1.0)

    basic = coo_matrix((data, (row, col)), shape=(valid.shape[0], 1983+9897))
    ext = valid.ix[:, 3:].fillna(0).values
    ext[:, 4] = ext[:, 4]/4
    ext[:, 10] = ext[:, 10]/4
    value = hstack([basic, ext])
    dump_svmlight_file(value, valid.rating.values, path+"libfm/valid_{0}.csv".format(cv))

In [8]:
from scipy.sparse import coo_matrix, hstack
from sklearn.datasets import dump_svmlight_file


path = "/mnt/d/Data/mangaki-data-challenge/"

train = pd.read_csv(path+"baseline/train_0.csv")
valid = pd.read_csv(path+"baseline/test_0.csv")

data = []
row = []
col = []
for i, itm in enumerate(train.iterrows()):
    row.append(i)
    col.append(itm[1].user_id)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].work_id+1983)
    data.append(1.0)

basic = coo_matrix((data, (row, col)), shape=(train.shape[0], 1983+9897))
ext = train.ix[:, 3:].fillna(0).values
ext[:, 4] = ext[:, 4]/4
ext[:, 10] = ext[:, 10]/4
value = hstack([basic, ext])
dump_svmlight_file(value, train.rating.values, path+"libfm/train_0.csv")

data = []
row = []
col = []
for i, itm in enumerate(valid.iterrows()):
    row.append(i)
    col.append(itm[1].user_id)
    data.append(1.0)
    row.append(i)
    col.append(itm[1].work_id+1983)
    data.append(1.0)

basic = coo_matrix((data, (row, col)), shape=(valid.shape[0], 1983+9897))
ext = valid.ix[:, 2:].fillna(0).values
ext[:, 4] = ext[:, 4]/4
ext[:, 10] = ext[:, 10]/4
value = hstack([basic, ext])
dump_svmlight_file(value, np.zeros(valid.shape[0]), path+"libfm/test_0.csv")

In [2]:
train = pd.read_csv(data+"baseline/train_0.csv")
valid = pd.read_csv(data+"baseline/test_0.csv")

In [5]:
train.head()

,user_id,work_id,rating,user_dislike,user_like,user_love,user_neutral,user_mean,user_std,work_dislike,work_like,work_love,work_neutral,item_mean,item_std
0,50,4041,0,0.391892,0.391892,0.040541,0.175676,2.081081,0.975828,0.074786,0.702991,0.042735,0.179487,2.713675,0.663571
1,508,1713,0,0.117895,0.564211,0.048421,0.269474,2.543158,0.762885,0.148936,0.425532,0.148936,0.276596,2.574468,0.926534
2,1780,7053,1,0.048701,0.681818,0.001623,0.267857,2.636364,0.576710,0.000000,0.333333,0.000000,0.666667,2.333333,0.516398
3,658,8853,0,0.098039,0.398693,0.431373,0.071895,3.163399,0.935116,0.153846,0.500000,0.076923,0.269231,2.500000,0.860233
4,1003,9401,0,0.000000,0.714286,0.000000,0.285714,2.714286,0.487950,0.179763,0.520990,0.037675,0.261572,2.416577,0.823801


In [6]:
valid.head()

,user_id,work_id,user_dislike,user_like,user_love,user_neutral,user_mean,user_std,work_dislike,work_like,work_love,work_neutral,item_mean,item_std
0,486,1086,0.119565,0.728261,0.0,0.152174,2.608696,0.694666,0.279412,0.225490,0.009804,0.485294,1.965686,0.738657
1,1509,3296,0.048276,0.841379,0.0,0.110345,2.793103,0.512301,0.121212,0.686869,0.010101,0.181818,2.585859,0.714430
2,617,1086,0.027778,0.833333,0.0,0.138889,2.805556,0.467177,0.279412,0.225490,0.009804,0.485294,1.965686,0.738657
3,270,9648,0.152597,0.629870,0.0,0.217532,2.477273,0.745979,0.062500,0.666667,0.027778,0.243056,2.659722,0.638652
4,459,3647,0.031373,0.450980,0.0,0.517647,2.419608,0.554516,0.134146,0.707317,0.024390,0.134146,2.621951,0.747675


Using lsi

In [21]:
num_topic=20

In [17]:
record_positive = record
record_positive_effectiveuser = record_positive[['user_id', 'work_id']].groupby(by='user_id')['work_id'].agg({'work_count': 'count'})
record_positive_effectiveuser = record_positive_effectiveuser[record_positive_effectiveuser.work_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='user_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveuser), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5, 'dislike':-1}
for itm in record_positive.iterrows():
    for work in itm[1].work_id:
        strwork = 'i{0}'.format(work)
        if strwork not in dictionary:
            dictionary[strwork]=cnt
            cnt+=1
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

In [14]:
from gensim.models.lsimodel import LsiModel

lsimodel = LsiModel(corpus, num_topics=num_topic)

In [15]:
record_positive = record.groupby(by='user_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['i{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].work_id, itm[1].rating) if 'i{0}'.format(i) in dictionary])

user_lsi_positive = []
for i, topic in enumerate(lsimodel[corpus]):
    temp = np.zeros(num_topic)
    for itm in topic:
        temp[itm[0]] = itm[1]
    user_lsi_positive.append(temp)
user_lsi_positive = np.vstack(user_lsi_positive)
u6 = pd.DataFrame(data = user_lsi_positive, index=record_positive.index, columns=['user_lsi_{0}'.format(i) for i in range(num_topic)])

u6.reset_index().to_csv(data+"features/user_lsi_{0}d.csv".format(num_topic), index=False)

In [7]:
u6.head()

,user_lsi_0,user_lsi_1,user_lsi_2,user_lsi_3,user_lsi_4,user_lsi_5,user_lsi_6,user_lsi_7,user_lsi_8,user_lsi_9
user_id,,,,,,,,,,
0,1.550102,0.184727,1.420607,-1.484637,-0.104344,-0.957708,0.300948,0.333724,-0.631171,-0.278322
1,3.175376,-0.852548,0.193455,-0.825350,-0.168458,-0.035739,0.353309,-0.141072,1.551080,0.187146
2,2.027113,-0.800671,0.495223,-0.685109,-0.011980,0.846306,0.165598,-0.043316,0.598268,0.641669
3,7.769884,-7.004141,1.018471,0.427918,-1.493721,0.710533,3.962426,-0.102615,2.698168,0.787107
4,5.456074,3.889428,-0.441991,-2.041339,-0.492546,0.685152,-0.064113,-0.539779,-0.639626,-0.064427


In [22]:
record_positive = record
record_positive_effectiveitem = record_positive[['user_id', 'work_id']].groupby(by='work_id')['user_id'].agg({'user_count': 'count'})
record_positive_effectiveitem = record_positive_effectiveitem[record_positive_effectiveitem.user_count>5]
record_positive = record_positive[['user_id', 'work_id', 'rating']].groupby(by='work_id').agg(lambda x: list(x))
record_positive = pd.concat((record_positive, record_positive_effectiveitem), axis=1, join='inner')

corpus = []
dictionary = {}
cnt=0
ratlut = {'love': 1.5, 'like': 1, 'neutral': 0.5, 'dislike':-1}
for itm in record_positive.iterrows():
    for user in itm[1].user_id:
        struser = 'u{0}'.format(user)
        if struser not in dictionary:
            dictionary[struser]=cnt
            cnt+=1
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating)])
invdict = dict(zip(dictionary.values(), dictionary.keys()))

lsimodel = LsiModel(corpus, num_topics=num_topic)

record_positive = record.groupby(by='work_id').agg(lambda x: list(x))

corpus = []
for itm in record_positive.iterrows():
    corpus.append([(dictionary['u{0}'.format(i)], ratlut[r]) for i,r in zip(itm[1].user_id, itm[1].rating) if 'u{0}'.format(i) in dictionary])

item_lsi_positive = []
for i, topic in enumerate(lsimodel[corpus]):
    temp = np.zeros(num_topic)
    for itm in topic:
        temp[itm[0]] = itm[1]
    item_lsi_positive.append(temp)
item_lsi_positive = np.vstack(item_lsi_positive)
i6 = pd.DataFrame(data = item_lsi_positive, index=record_positive.index, columns=['work_lsi_{0}'.format(i) for i in range(num_topic)])

i6.reset_index().to_csv(data+"features/item_lsi_{0}d.csv".format(num_topic), index=False)

In [20]:
i6

,work_lsi_0,work_lsi_1,work_lsi_2,work_lsi_3,work_lsi_4,work_lsi_5,work_lsi_6,work_lsi_7,work_lsi_8,work_lsi_9
work_id,,,,,,,,,,
0,0.085805,-0.012528,-0.101966,0.080332,-0.005967,0.041260,-0.029676,-0.041790,0.066928,-0.094275
1,0.070006,-0.083790,0.074429,0.067920,0.018778,0.042432,-0.113090,-0.015373,-0.033495,-0.083969
2,12.269769,4.599712,-0.011665,-1.458024,6.681542,-4.024264,0.134369,-3.460256,4.324890,5.313904
3,0.097232,0.138147,0.114934,0.228180,0.021407,0.179143,0.078076,-0.057941,-0.122832,0.072458
4,0.416710,-0.479190,0.068844,0.079159,0.711638,0.022947,0.296789,-0.119784,-0.531556,-0.341670
5,1.973421,-1.402326,0.260394,0.916047,0.916707,0.668729,-0.292848,0.639267,-0.320131,0.207612
6,0.018153,-0.004152,0.026293,-0.004238,0.013172,-0.012732,0.010480,0.004824,-0.013551,-0.008159
7,0.125776,-0.043919,0.076601,0.020317,0.257997,-0.068892,0.018711,-0.039423,-0.024054,-0.041383
9,0.994882,-0.662120,-0.070725,0.269806,0.508631,-0.079590,0.315049,-0.715961,-0.023754,-0.635282
